<a href="https://colab.research.google.com/github/iSanjayYadava/deeplearning-models/blob/master/perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline

In [0]:
import torch
import numpy as np

In [5]:
device = ("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#from pathlib import Path
#DATA_PATH = Path('/content/gdrive/My\ Drive/data/perceptron_toydata.txt')


In [8]:
data = np.genfromtxt('/content/gdrive/My Drive/data/perceptron_toydata.txt', delimiter='\t')
X , y = data[:,:2] , data[:,2]
y = y.astype(int)
print("X.shape",X.shape)
print('y.shape',y.shape)
print('Label count',np.bincount(y))

X.shape (100, 2)
y.shape (100,)
Label count [50 50]


In [0]:
# Shuffling & train/test split
shuffle_idx = np.arange(y[0])
shuffle_rnd = np.random.RandomState(123)
shuffle_rnd.shuffle(shuffle_idx)

X_train,X_test = X[shuffle_idx[:70]], X[shuffle_idx[70:]]
y_train,y_test = y[shuffle_idx[:70]], X[shuffle_idx[70:]]

In [0]:
def custom_where(cond, x_1, x_2):
    return (cond * x_1) + ((1-cond) * x_2)

In [0]:
class Perceptron():
  
  def __init__(self,num_features):
    self.num_features = num_features
    self.weights = torch.zeros(num_features,1,dtype=torch.float32,device=device)
    self.bias = torch.zeros(1,dtype=torch.float32,device= device)
    
  def forward(self,x):
    linear = torch.add(torch.mm(x,self.weights),self.bias)
    predictions = custom_where(linear > 0., 1, 0).float()
    return predictions
  
  def backward(self , x ,y):
    predictions = self.forward(x)
    errors = y - predictions
    return errors
  
  def evaluate(self, x, y):
    predictions = self.forward(x).view(-1)
    accuracy = torch.sum(y == predictions).float() / y.shape()[0]
    return accuracy

  def train(self, x, y, epochs):
    for e in range(epochs):
      for i in range(y.size()[0]):
        errors = self.backward(x[i].view(1, self.num_features), y[i]).view(-1)
        self.weights += (errors * x[i]).view(self.num_features, 1)
        self.bias += errors

In [12]:
# Training 
ppn = Perceptron(num_features=2)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32, device=device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32, device=device)

ppn.train(X_train_tensor, y_train_tensor, epochs=10)

print('Model parameters:')
print('  Weights: %s' % ppn.weights)
print('  Bias: %s' % ppn.bias)



Model parameters:
  Weights: tensor([[0.],
        [0.]], device='cuda:0')
  Bias: tensor([0.], device='cuda:0')


In [13]:


X_test_tensor = torch.tensor(X_test, dtype=torch.float32, device=device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32, device=device)

test_acc = ppn.evaluate(X_test_tensor, y_test_tensor)
print('Test set accuracy: %.2f%%' % (test_acc*100))



RuntimeError: ignored